In [47]:
import pandas as pd
from datetime import datetime, timedelta

In [48]:
cesta_k_suboru = '/content/RozsAgloSP4-2018-vystup.xlsx'
nazov_listu = 'Data'
dataset = pd.read_excel(cesta_k_suboru, sheet_name=nazov_listu)

In [49]:
upravene_casy = []
# Prechádzanie cez všetky pôvodné dátumy a časy meraní
for cas in dataset['Dátum a čas']:

    cas_str = cas.strftime("%Y-%m-%d %H:%M:%S")
    datum_cas = datetime.strptime(cas_str, "%Y-%m-%d %H:%M:%S")

    # Upravenie času merania
    if datum_cas.hour >= 23:
        datum_cas = datum_cas.replace(hour=23, minute=0)
    elif datum_cas.hour >= 11:
        datum_cas = datum_cas.replace(hour=11, minute=0)
    else:
        datum_cas -= timedelta(days=1)
        datum_cas = datum_cas.replace(hour=23, minute=0)
    # Pridanie upraveného dátumu a času do nového zoznamu
    upravene_casy.append(datum_cas.strftime("%Y-%m-%d %H:%M"))

dataset['Upravené časy intervalov'] = upravene_casy

#for cas in upravene_casy:
#    print(cas)

# Konvertovanie stĺpca 'Upravené časy intervalov' na typ datetime
dataset['Upravené časy intervalov'] = pd.to_datetime(dataset['Upravené časy intervalov'])

osobitny_zoznam = []
skupiny = dataset.groupby('Upravené časy intervalov')

for _, skupina in skupiny:
    # Získanie dátumu a času pre prvý riadok v skupine
    prvy_cas = skupina.iloc[0]['Dátum a čas']
    najblizsi_cas = prvy_cas

    for _, riadok in skupina.iterrows():
        # Získanie dátumu a času pre aktuálny riadok
        aktualny_cas = riadok['Dátum a čas']
        # Porovnanie súčasného času s najbližším nájdeným časom
        if abs((aktualny_cas - riadok['Upravené časy intervalov']).total_seconds()) < abs((najblizsi_cas - riadok['Upravené časy intervalov']).total_seconds()):
            najblizsi_cas = aktualny_cas
    # Pridanie najbližšieho času do osobitného zoznamu
    osobitny_zoznam.append(najblizsi_cas)

# Filtrácia riadkov, ktorých čas je zhodný s časmi v zozname osobitny_zoznam
upravene_data = dataset[dataset['Dátum a čas'].isin(osobitny_zoznam)]
upravene_data = upravene_data.reset_index(drop=True)

#for cas in osobitny_zoznam:
#    print(cas)

In [50]:
len(osobitny_zoznam)

502

In [51]:
upravene_data.count()


Dátum a čas                 502
Zmena                       502
8hod                        502
12hod                       502
SP                          502
Vzorka                      502
Zhoda                       502
0-5                         502
2024-10-05 00:00:00         502
10-25                       502
nad 25                      502
nad 10                      502
ISO Zhoda                     3
ISO 0-5                       3
ISO > 6,3                     3
Upravené časy intervalov    502
dtype: int64

In [55]:
upravene_data

,Dátum a čas,Zmena,8hod,12hod,SP,Vzorka,Zhoda,0-5,2024-10-05 00:00:00,10-25,nad 25,nad 10,ISO Zhoda,ISO 0-5,"ISO > 6,3",Upravené časy intervalov
0,2018-12-31 12:08:18,1,1D,R7,4,41679,A,10.71,21.43,42.50,25.36,67.86,NaN,NaN,NaN,2018-12-31 11:00:00
1,2018-12-30 23:56:11,3,3E,N9,4,41676,A,6.90,20.69,37.93,34.48,72.41,NaN,NaN,NaN,2018-12-30 23:00:00
2,2018-12-30 12:43:11,1,1D,R8,4,41673,A,8.05,22.88,38.98,30.08,69.06,NaN,NaN,NaN,2018-12-30 11:00:00
3,2018-12-30 00:15:03,3,3B,N5,4,41670,A,6.21,18.63,46.58,28.57,75.15,NaN,NaN,NaN,2018-12-29 23:00:00
4,2018-12-29 12:23:16,1,1A,R9,4,41667,A,4.21,18.95,42.11,34.74,76.85,NaN,NaN,NaN,2018-12-29 11:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,2018-01-03 01:58:37,3,3D,N6,4,40006,A,7.04,18.78,40.61,33.57,74.18,NaN,NaN,NaN,2018-01-02 23:00:00
498,2018-01-02 11:30:07,1,1B,R5,4,40004,N,36.17,21.28,26.60,15.96,42.56,NaN,NaN,NaN,2018-01-02 11:00:00
499,2018-01-02 02:02:57,3,3D,N7,4,40003,A,11.62,30.71,29.05,28.63,57.68,NaN,NaN,NaN,2018-01-01 23:00:00
500,2018-01-01 16:49:22,2,2C,R6,4,40002,A,9.46,16.55,40.19,33.80,73.99,NaN,NaN,NaN,2018-01-01 11:00:00


In [56]:
upravene_data.to_excel('/content/RozsAgloSP4-2018-upraveny.xlsx', index=False)

In [57]:
cesta_k_suboru = '/content/RozsAgloSP4-2018-upraveny.xlsx'
data = pd.read_excel(cesta_k_suboru, 'Sheet1')

data

,Dátum a čas,Zmena,8hod,12hod,SP,Vzorka,Zhoda,0-5,2024-10-05 00:00:00,10-25,nad 25,nad 10,ISO Zhoda,ISO 0-5,"ISO > 6,3",Upravené časy intervalov
0,2018-12-31 12:08:18,1,1D,R7,4,41679,A,10.71,21.43,42.50,25.36,67.86,NaN,NaN,NaN,2018-12-31 11:00:00
1,2018-12-30 23:56:11,3,3E,N9,4,41676,A,6.90,20.69,37.93,34.48,72.41,NaN,NaN,NaN,2018-12-30 23:00:00
2,2018-12-30 12:43:11,1,1D,R8,4,41673,A,8.05,22.88,38.98,30.08,69.06,NaN,NaN,NaN,2018-12-30 11:00:00
3,2018-12-30 00:15:03,3,3B,N5,4,41670,A,6.21,18.63,46.58,28.57,75.15,NaN,NaN,NaN,2018-12-29 23:00:00
4,2018-12-29 12:23:16,1,1A,R9,4,41667,A,4.21,18.95,42.11,34.74,76.85,NaN,NaN,NaN,2018-12-29 11:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,2018-01-03 01:58:37,3,3D,N6,4,40006,A,7.04,18.78,40.61,33.57,74.18,NaN,NaN,NaN,2018-01-02 23:00:00
498,2018-01-02 11:30:07,1,1B,R5,4,40004,N,36.17,21.28,26.60,15.96,42.56,NaN,NaN,NaN,2018-01-02 11:00:00
499,2018-01-02 02:02:57,3,3D,N7,4,40003,A,11.62,30.71,29.05,28.63,57.68,NaN,NaN,NaN,2018-01-01 23:00:00
500,2018-01-01 16:49:22,2,2C,R6,4,40002,A,9.46,16.55,40.19,33.80,73.99,NaN,NaN,NaN,2018-01-01 11:00:00


In [58]:
cesta_k_suboru1 = '/content/PI_DataLink_Aglo_digital twins-5min-SP4-2018-15minute.xlsm'
nazov_listu1 = 'Sheet1'
data1 = pd.read_excel(cesta_k_suboru1, sheet_name=nazov_listu1)

data1 = data1.iloc[3:, 2:]
data1 = data1.reset_index(drop=True)
data1

,Unnamed: 2,Unnamed: 3,výstupné dáta,výstupné dáta.1,výstupné dáta.2,"výstupné dáta, látkové filtre - pred komínom","vstupné dáta, 3 stupen miesania, reguluje palič","vstupné dáta, reguluje palič","vstupné dáta, 3 stupeň","vstupné dáta, reguluje palič.1",...,výstupné dáta.10,výstupné dáta.11,výstupné dáta.12,výstupné dáta.13,výstupné dáta.14,výstupné dáta.15,vstupné dáta.1,"vstupné dáta, reguluje palič.12",vstupné dáta.2,vstupné dáta.3
0,2018-01-01 10:00:00,2018-01-01 10:15:00,18.264017,700.266782,24.1226,0,1,228.719401,0,3.779417,...,168.547434,176.211767,185.319134,202.393551,166.576,116.467117,0.7589,2.008533,1329.989014,1329.990013
1,2018-01-01 10:15:00,2018-01-01 10:30:00,18.2956,706.237531,24.534617,0,1,228.235471,0,3.77955,...,165.095049,175.516066,176.825516,196.314184,165.274332,113.7811,0.751883,1.99735,1329.995504,1329.996503
2,2018-01-01 10:30:00,2018-01-01 10:45:00,18.10195,700.206297,26.535567,0,1,227.910886,0,3.792367,...,153.602331,166.481549,175.229049,193.549751,159.180799,112.111283,0.740383,1.983817,1329.996004,1329.983012
3,2018-01-01 10:45:00,2018-01-01 11:00:00,18.124149,700.073284,26.7562,0,1,228.238816,0,3.766617,...,164.521083,176.025567,188.248565,211.764735,175.968851,124.459334,0.7403,1.982783,1329.99101,1329.989506
4,2018-01-01 11:00:00,2018-01-01 11:15:00,18.245233,708.250231,26.974017,0,1,227.713533,0,3.7711,...,165.478184,166.134733,196.306917,220.87265,192.920016,129.854734,0.740317,1.979583,1329.998008,1329.99101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34947,2018-12-31 10:45:00,2018-12-31 11:00:00,18.816999,708.57769,77.524151,0,1,227.62555,0,4.1584,...,128.908483,128.110566,113.692766,164.451517,184.53605,154.147232,0.71165,2.0372,1360.043502,1360.002511
34948,2018-12-31 11:00:00,2018-12-31 11:15:00,18.816999,707.584498,80.058234,0,1,228.281668,0,4.150883,...,132.293666,131.06595,120.070049,163.032982,176.629483,146.382766,0.711367,2.037433,1360.023505,1360.015007
34949,2018-12-31 11:15:00,2018-12-31 11:30:00,18.816999,708.969909,82.582333,0,1,228.045238,0,4.149867,...,143.4212,146.387684,134.3365,184.606716,196.979168,158.829367,0.711333,2.0374,1360.039018,1360.010006
34950,2018-12-31 11:30:00,2018-12-31 11:45:00,18.816999,711.833297,76.172483,0,1,182.81985,0,3.613983,...,149.56125,153.987266,144.052017,194.894467,204.016834,163.083466,0.6025,1.713833,1360.031012,1359.989502


In [59]:
data1.count()

Unnamed: 2                                         34952
Unnamed: 3                                         34952
výstupné dáta                                      34952
výstupné dáta.1                                    34952
výstupné dáta.2                                    34952
výstupné dáta, látkové filtre - pred komínom       34952
vstupné dáta, 3 stupen miesania, reguluje palič    34952
vstupné dáta, reguluje palič                       34952
vstupné dáta, 3 stupeň                             34952
vstupné dáta, reguluje palič.1                     34952
Unnamed: 12                                        34952
Unnamed: 13                                        34952
Unnamed: 14                                        34952
vstupné dáta, reguluje palič.2                     34952
Unnamed: 16                                        34952
vstupné dáta                                       34952
výstupné dáta.3                                    34952
výstupné dáta.4                

In [60]:
data['Dátum'] = pd.to_datetime(data['Upravené časy intervalov']).dt.date
data1['Dátum'] = pd.to_datetime(data1['Unnamed: 2']).dt.date

# Nájdenie zhodujúcich dátumov
result = data1[data1['Dátum'].isin(data['Dátum'])]

# Premenovanie stĺpcov
result = result.rename(columns={'Unnamed: 2': 'Interval od', 'Unnamed: 3': 'Interval do'})
result = result.drop_duplicates(subset=['Interval od'])

# Vytvorenie nových stĺpcov v DataFrame result
result['Zapis merania'] = None
result['Vykonanie merania'] = None
result['Vzorka'] = None
result['Zhoda'] = None
result['Rozsev 0-5'] = None

for index, row in result.iterrows():
    interval_od = row['Interval od']
    interval_do = row['Interval do']
    mask = (data['Upravené časy intervalov'] >= interval_od) & (data['Upravené časy intervalov'] < interval_do)
    filtered_data = data[mask]
    if not filtered_data.empty:
        result.loc[index, 'Zapis merania'] = filtered_data['Dátum a čas'].iloc[0]
        result.loc[index, 'Vykonanie merania'] = filtered_data['Upravené časy intervalov'].iloc[0]
        result.loc[index, 'Vzorka'] = filtered_data['Vzorka'].iloc[0]
        result.loc[index, 'Zhoda'] = filtered_data['Zhoda'].iloc[0]
        result.loc[index, 'Rozsev 0-5'] = filtered_data['0-5'].iloc[0]

#Odstránenie nepotrebných stlpcov
result = result.drop('Dátum', axis=1)
result = result.drop('výstupné dáta, látkové filtre - pred komínom', axis=1)
result = result.drop('vstupné dáta, 3 stupeň', axis=1)
result = result.drop('Unnamed: 12', axis=1)
result = result.drop('Unnamed: 13', axis=1)
result = result.drop('Unnamed: 14', axis=1)
result = result.drop('Unnamed: 16', axis=1)
result = result.drop('výstupné dáta.3', axis=1)
result = result.drop('výstupné dáta.4', axis=1)
result = result.drop('Unnamed: 27', axis=1)
result = result.drop('vstupné dáta, reguluje palič.10', axis=1) #No Good Data For Calculation
result = result.drop('výstupné dáta.5', axis=1)
result = result.drop('Unnamed: 30', axis=1)
result = result.drop('vstupné dáta.2', axis=1)
result = result.drop('vstupné dáta.3', axis=1)

result = result.reset_index(drop=True)
vystupny_subor = '/content/vystup.xlsx'
result.to_excel(vystupny_subor, index=False)

<ipython-input-60-9eaa5d830c92>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['Zapis merania'] = None
<ipython-input-60-9eaa5d830c92>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['Vykonanie merania'] = None
<ipython-input-60-9eaa5d830c92>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

In [61]:
result

,Interval od,Interval do,výstupné dáta,výstupné dáta.1,výstupné dáta.2,"vstupné dáta, 3 stupen miesania, reguluje palič","vstupné dáta, reguluje palič","vstupné dáta, reguluje palič.1","vstupné dáta, reguluje palič.2",vstupné dáta,...,výstupné dáta.13,výstupné dáta.14,výstupné dáta.15,vstupné dáta.1,"vstupné dáta, reguluje palič.12",Zapis merania,Vykonanie merania,Vzorka,Zhoda,Rozsev 0-5
0,2018-01-01 10:00:00,2018-01-01 10:15:00,18.264017,700.266782,24.1226,1,228.719401,3.779417,6.294617,6.476267,...,202.393551,166.576,116.467117,0.7589,2.008533,None,None,None,None,None
1,2018-01-01 10:15:00,2018-01-01 10:30:00,18.2956,706.237531,24.534617,1,228.235471,3.77955,6.0343,7.050283,...,196.314184,165.274332,113.7811,0.751883,1.99735,None,None,None,None,None
2,2018-01-01 10:30:00,2018-01-01 10:45:00,18.10195,700.206297,26.535567,1,227.910886,3.792367,6.369767,6.833633,...,193.549751,159.180799,112.111283,0.740383,1.983817,None,None,None,None,None
3,2018-01-01 10:45:00,2018-01-01 11:00:00,18.124149,700.073284,26.7562,1,228.238816,3.766617,6.479617,6.008617,...,211.764735,175.968851,124.459334,0.7403,1.982783,None,None,None,None,None
4,2018-01-01 11:00:00,2018-01-01 11:15:00,18.245233,708.250231,26.974017,1,227.713533,3.7711,6.3795,5.953333,...,220.87265,192.920016,129.854734,0.740317,1.979583,2018-01-01 16:49:22,2018-01-01 11:00:00,40002,A,9.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27167,2018-12-31 10:45:00,2018-12-31 11:00:00,18.816999,708.57769,77.524151,1,227.62555,4.1584,4.769,6.86,...,164.451517,184.53605,154.147232,0.71165,2.0372,None,None,None,None,None
27168,2018-12-31 11:00:00,2018-12-31 11:15:00,18.816999,707.584498,80.058234,1,228.281668,4.150883,4.769,6.86,...,163.032982,176.629483,146.382766,0.711367,2.037433,2018-12-31 12:08:18,2018-12-31 11:00:00,41679,A,10.71
27169,2018-12-31 11:15:00,2018-12-31 11:30:00,18.816999,708.969909,82.582333,1,228.045238,4.149867,4.769,6.86,...,184.606716,196.979168,158.829367,0.711333,2.0374,None,None,None,None,None
27170,2018-12-31 11:30:00,2018-12-31 11:45:00,18.816999,711.833297,76.172483,1,182.81985,3.613983,4.769,6.86,...,194.894467,204.016834,163.083466,0.6025,1.713833,None,None,None,None,None


In [62]:
result.count()

Interval od                                        27172
Interval do                                        27172
výstupné dáta                                      27172
výstupné dáta.1                                    27172
výstupné dáta.2                                    27172
vstupné dáta, 3 stupen miesania, reguluje palič    27172
vstupné dáta, reguluje palič                       27172
vstupné dáta, reguluje palič.1                     27172
vstupné dáta, reguluje palič.2                     27172
vstupné dáta                                       27172
vstupné dáta, reguluje palič.3                     27172
vstupné dáta, reguluje palič.4                     27172
vstupné dáta, reguluje palič.5                     27172
vstupné dáta, reguluje palič.6                     27172
vstupné dáta, reguluje palič.7                     27172
vstupné dáta, reguluje palič.8                     27172
vstupné dáta, reguluje palič.9                     27172
vstupné dáta, reguluje palič.11

In [63]:
subor = pd.read_excel(vystupny_subor)
subor = subor[['Interval od', 'Interval do','Zapis merania', 'Vykonanie merania', 'Vzorka', 'Zhoda', 'Rozsev 0-5']]
meranie_interval = pd.concat([subor, data1['výstupné dáta.5']], axis=1)
meranie_interval.to_excel('rozsev_v_intervaloch.xlsx', index=False)

In [64]:
# Odstránenie určených stĺpcov z result a zanedbanie indexovania
vysledny_dataset = result.drop(columns=['Interval od', 'Interval do', 'Zapis merania', 'Vykonanie merania', 'Vzorka', 'Zhoda'])
vysledny_dataset.dropna(inplace=True)

In [65]:
vysledny_dataset.drop_duplicates(subset=['Rozsev 0-5'])
vysledny_dataset = vysledny_dataset.reset_index(drop=True)

In [66]:
# Uloženie do súboru
vysledny_dataset.to_excel("vysledny_dataset.xlsx", index=False)

In [67]:
vysledny_dataset

,výstupné dáta,výstupné dáta.1,výstupné dáta.2,"vstupné dáta, 3 stupen miesania, reguluje palič","vstupné dáta, reguluje palič","vstupné dáta, reguluje palič.1","vstupné dáta, reguluje palič.2",vstupné dáta,"vstupné dáta, reguluje palič.3","vstupné dáta, reguluje palič.4",...,výstupné dáta.9,výstupné dáta.10,výstupné dáta.11,výstupné dáta.12,výstupné dáta.13,výstupné dáta.14,výstupné dáta.15,vstupné dáta.1,"vstupné dáta, reguluje palič.12",Rozsev 0-5
0,18.245233,708.250231,26.974017,1,227.713533,3.7711,6.3795,5.953333,3.22,-4.185,...,121.342817,165.478184,166.134733,196.306917,220.87265,192.920016,129.854734,0.740317,1.979583,9.46
1,17.924417,718.155392,31.431067,1,227.747202,3.567233,4.02235,8.63545,3.13,-4.577833,...,116.6768,161.837916,161.375434,186.621433,214.378767,186.251648,134.829633,0.761467,2.018433,11.62
2,19.169966,709.452829,161.9624,0,226.832703,3.276133,5.795817,8.048333,3,-4.9038,...,71.735,101.088067,94.693117,112.700217,125.127517,105.031466,70.711683,0.765667,2.047617,36.17
3,18.356667,709.410084,61.53855,1,227.996302,3.473383,5.867217,8.040933,3.09,-4.315217,...,139.248766,177.915884,176.855198,204.440951,234.291532,208.389883,140.35035,0.748317,2.00315,7.04
4,18.993284,715.636641,78.977666,0.863333,190.246086,2.574417,3.56085,7.290917,2.86,-4.38765,...,123.216516,156.741516,138.683849,168.911166,180.099066,156.663517,105.52515,0.702717,1.9422,7.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,18.816999,710.190049,81.033282,1,227.547802,4.6434,4.769,6.86,3.9,-3.463567,...,171.417384,156.030383,151.898268,128.774884,170.230866,177.4666,147.93215,0.66065,1.824967,4.21
497,18.816999,706.556821,72.524867,1,228.063869,4.7744,4.769,6.86,3.81,-3.551733,...,141.585617,147.590018,152.835733,134.094401,177.335335,183.941018,147.457733,0.7289,2.113767,6.21
498,18.816999,695.341931,64.947766,1,227.116967,4.442967,4.769,6.86,3.67,-3.978633,...,141.632734,132.0885,132.520417,117.497401,166.51765,184.331782,161.278251,0.712333,2.08055,8.05
499,18.816999,707.283079,77.876267,1,226.86465,4.131533,4.769,6.86,3.53,-3.697167,...,137.372949,132.912383,142.0003,139.437549,204.874482,220.287483,200.544516,0.749917,2.152633,6.9


In [68]:
vysledny_dataset.count()

výstupné dáta                                      501
výstupné dáta.1                                    501
výstupné dáta.2                                    501
vstupné dáta, 3 stupen miesania, reguluje palič    501
vstupné dáta, reguluje palič                       501
vstupné dáta, reguluje palič.1                     501
vstupné dáta, reguluje palič.2                     501
vstupné dáta                                       501
vstupné dáta, reguluje palič.3                     501
vstupné dáta, reguluje palič.4                     501
vstupné dáta, reguluje palič.5                     501
vstupné dáta, reguluje palič.6                     501
vstupné dáta, reguluje palič.7                     501
vstupné dáta, reguluje palič.8                     501
vstupné dáta, reguluje palič.9                     501
vstupné dáta, reguluje palič.11                    501
výstupné dáta.6                                    501
výstupné dáta.7                                    501
výstupné d